# Ball tracking
https://www.pyimagesearch.com/2015/09/14/ball-tracking-with-opencv/

In [1]:
from collections import deque
from imutils.video import VideoStream
import numpy as np
import cv2
import imutils
import time
import matplotlib.pyplot as plt

## Configuration

In [2]:
video_path = "videos/ball_tracking_example.mp4"
buffer_size = 64

# Define the lower and upper boundaries of the "green" ball in the HSV color space,
# then initialize the list of tracked points
greenLower = (29, 86, 6)
greenUpper = (64, 255, 255)
pts = deque(maxlen=buffer_size)

# Grab a reference to the video file
vs = cv2.VideoCapture(video_path)

# Allow the camera or video file to warm up
time.sleep(2.0)

## Process video

In [3]:
# Keep looping
while True:
    # Grab the current frame
    frame = vs.read()[1]
    
    # If we are viewing a video and we did not grab a frame,
    # then we have reached the end of the video
    if frame is None:
        break
        
    # Resize the frame, blur it, and convert it to the HSV color space
    frame = imutils.resize(frame, width=600)
    blurred = cv2.GaussianBlur(frame, (11, 11), 0)
    hsv = cv2.cvtColor(blurred, cv2.COLOR_BGR2HSV)
    
    # Construct a mask for the color "green", then perform a series of
    # dilations and erosions to remove any small blobs left in the mask
    mask = cv2.inRange(hsv, greenLower, greenUpper)
    mask = cv2.erode(mask, None, iterations=2)
    mask = cv2.dilate(mask, None, iterations=2)
    
    # Find contours in the mask and initialize the current (x, y) center of the ball
    cnts = cv2.findContours(mask.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    cnts = imutils.grab_contours(cnts)
    center = None
    
    # Only proceed if at least one contour was found
    if len(cnts) > 0:
        # Find the largest contour in the mask, then use it to compute
        # the minimum enclosing circle and centroid
        c = max(cnts, key=cv2.contourArea)
        ((x, y), radius) = cv2.minEnclosingCircle(c)
        M = cv2.moments(c)
        center = (int(M["m10"] / M["m00"]), int(M["m01"] / M["m00"]))
        
        # Only proceed if the radius meets a minimum size
        if radius > 10:
            # Draw the circle and centroid on the frame, then update the list of tracked points
            cv2.circle(frame, (int(x), int(y)), int(radius), (0, 255, 255), 2)
            cv2.circle(frame, center, 5, (0, 0, 255), -1)
        
        # Update the points queue
        pts.appendleft(center)
        
    # Loop over the set of tracked points
    for i in range(1, len(pts)):
        # If either of the tracked points are None, ignore them
        if pts[i - 1] is None or pts[i] is None:
            continue
            
        # Otherwise, compute the thickness of the line and draw the connecting lines
        thickness = int(np.sqrt(buffer_size / float(i + 1)) * 2.5)  # Thicker for closer locations
        cv2.line(frame, pts[i - 1], pts[i], (0, 0, 255), thickness)
        
    # Show the frame to our screen
    cv2.imshow("Frame", frame)
    key = cv2.waitKey(1) & 0xFF

    # If the 'q' key is pressed, stop the loop
    if key == ord("q"):
        break

vs.release()
cv2.destroyAllWindows()